# Multiple Choice Commonsense Reasoning - SocialIQA

> Fine-tuning Pre-trained Language Models on Multiple Choice Question Answering for Commonsense Reasoning 

> Based on HuggingFace Transformers

> Chaehyeong Kim, CONVEI Lab 

In [ ]:
!pip install transformers==4.26.1 evaluate datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

## 1. Load Library

In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate

from tqdm.auto import tqdm

## Set Hyperparameters

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [ ]:
BASE_DIR = os.getcwd()
OUTPUT_DIR = os.path.join(BASE_DIR, 'checkpoint')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
MODEL_NAME = 'bert-base-uncased'

## Load Tokenizer and Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMultipleChoice.from_pretrained(MODEL_NAME).to(DEVICE)

In [ ]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

## 3. Load Data

In [ ]:
datasets = load_dataset('social_i_qa')

Generating train split:   0%|          | 0/33410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1954 [00:00<?, ? examples/s]

Dataset social_i_qa downloaded and prepared to /root/.cache/huggingface/datasets/social_i_qa/default/0.1.0/674d85e42ac7430d3dcd4de7007feaffcb1527c535121e09bab2803fbcc925f8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answerA', 'answerB', 'answerC', 'label'],
        num_rows: 33410
    })
    validation: Dataset({
        features: ['context', 'question', 'answerA', 'answerB', 'answerC', 'label'],
        num_rows: 1954
    })
})

In [ ]:
train_dataset = load_dataset('social_i_qa', split='train[:70%]')
valid_dataset = load_dataset('social_i_qa', split='train[70%:]')
test_dataset = load_dataset('social_i_qa', split='validation')

In [ ]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

23387
10023
1954


In [ ]:
train_dataset[0]

{'context': 'Cameron decided to have a barbecue and gathered her friends together.',
 'question': 'How would Others feel as a result?',
 'answerA': 'like attending',
 'answerB': 'like staying home',
 'answerC': 'a good friend to have',
 'label': '1'}

In [ ]:
from datasets import ClassLabel
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets['train'])

,context,question,answerA,answerB,answerC,label
0,"Aubrey bought a large chocolate cake for her birthday, ate the whole thing by herself, and then ordered a pizza.",How would you describe Aubrey?,disciplined,lazy,ravenous,3
1,Kendall had her friends over for dinner and gave them a warm welcome when they arrived.,How would Others feel as a result?,like a good hostess,excited about the dinner,accepted,3
2,Carson decided he wanted a new start and moved to another state.,How would Carson feel afterwards?,scared to start fresh,ready to start over,tired of the old place,2
3,Austin did not bring a bike. Austin rode Jesse's bike.,How would Austin feel afterwards?,unaware,thrifty,indebted,3
4,Tracy went to work with a friend on selling computers door to door.,What will Tracy want to do next?,rest on her laurels,learn more about the industry,break away from the partnership,2
5,"Out of breath and winded, Tracy finally came within sight not far off from the finish line.",How would Tracy feel afterwards?,burdened by the last hurdle,committed,near,2
6,"In a photo editing app, Cameron was being silly, and Cameron turned Taylor's face upside down.",What will Taylor want to do next?,throw the photo away,show the photo to Tyler,burn the photo into ash,1
7,"After making a deal and almost signing with one company, Riley accepted another offer.",What will Riley want to do next?,Apologise to first company,Pretend like nothing happened,apply for multiple jobs,1
8,Remy spent the night in the hospital. They had a heart attack.,How would Remy feel afterwards?,gravely ill,glad to be taken care off,much worse,2
9,Sydney turned red when Carl cracked a sex joke and stared at her.,How would you describe Sydney?,Sidney did not like the sexual attention and wanted to have sex with him that night,embarrassed,Sidney did not like the sexual attention which she thought premature,2


## Preprocess Data

In [ ]:
answer_names = ['answerA', 'answerB', 'answerC']
num_choices = len(answer_names)

In [ ]:
def preprocess_function(examples):
    num_examples = len(examples['context'])
    # Repeat each first sentence three times (=num_choices) to go with the three possibilities of second sentences.
    first_sentences = [[examples['context'][i] + tokenizer.sep_token + examples['question'][i]] * num_choices for i in range(num_examples)]
    second_sentences = [[examples[answer][i] for answer in answer_names] for i in range(num_examples)]
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i : i + num_choices] for i in range(0, len(v), num_choices)] for k, v in tokenized_examples.items()}

In this function, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (3) and a list of input IDs (length varying here since we did not apply any padding)

In [ ]:
examples = datasets['train'][:5]
features = preprocess_function(examples)
print(len(features['input_ids']), len(features['input_ids'][0]), [len(x) for x in features['input_ids'][0]])

5 3 [26, 27, 29]


In [ ]:
idx = 0
[tokenizer.decode(features['input_ids'][idx][i]) for i in range(num_choices)]

['[CLS] cameron decided to have a barbecue and gathered her friends together. [SEP] how would others feel as a result? [SEP] like attending [SEP]',
 '[CLS] cameron decided to have a barbecue and gathered her friends together. [SEP] how would others feel as a result? [SEP] like staying home [SEP]',
 '[CLS] cameron decided to have a barbecue and gathered her friends together. [SEP] how would others feel as a result? [SEP] a good friend to have [SEP]']

In [ ]:
tokenized_train_data = train_dataset.map(preprocess_function, batched=True)
tokenized_valid_data = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/23387 [00:00<?, ? examples/s]

Map:   0%|          | 0/10023 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [int(feature.pop(label_name))-1 for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

When called on a list of examples, it will flatten all the input_ids/attention_mask in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * num_choices) x seq_length`) that we then unflatten.

In [ ]:
accepted_keys = ['input_ids', 'attention_mask', 'label']
features = [{k: v for k, v in tokenized_train_data[i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

In [ ]:
[tokenizer.decode(batch['input_ids'][8][i].tolist()) for i in range(3)]

['[CLS] quinn wanted to help me clean my room up because it was so messy. [SEP] what will quinn want to do next? [SEP] eat messy snacks [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] quinn wanted to help me clean my room up because it was so messy. [SEP] what will quinn want to do next? [SEP] help out a friend [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] quinn wanted to help me clean my room up because it was so messy. [SEP] what will quinn want to do next? [SEP] pick up the dirty clothes [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

## Train Model

In [ ]:
accuracy = evaluate.load('accuracy')

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir = True,
    do_train=True,
    do_eval=True,
    evaluation_strategy='steps',
    save_strategy='steps',
    logging_strategy='steps',
    eval_steps=200,
    save_steps=200,
    logging_steps=20,
    save_total_limit=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    max_grad_norm=0.1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    seed=516,
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_valid_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: answerC, answerA, answerB, question, context. If answerC, answerA, answerB, question, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 23387
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2193
  Number of trainable parameters = 109483009


{'loss': 1.1277, 'learning_rate': 4.997720018239854e-05, 'epoch': 0.0}
{'loss': 1.0982, 'learning_rate': 4.954400364797081e-05, 'epoch': 0.03}
{'loss': 1.0244, 'learning_rate': 4.908800729594164e-05, 'epoch': 0.05}
{'loss': 0.9792, 'learning_rate': 4.863201094391245e-05, 'epoch': 0.08}
{'loss': 0.9231, 'learning_rate': 4.8176014591883265e-05, 'epoch': 0.11}
{'loss': 0.8773, 'learning_rate': 4.772001823985408e-05, 'epoch': 0.14}
{'loss': 0.9189, 'learning_rate': 4.72640218878249e-05, 'epoch': 0.16}
{'loss': 0.8316, 'learning_rate': 4.680802553579571e-05, 'epoch': 0.19}
{'loss': 0.8826, 'learning_rate': 4.6352029183766534e-05, 'epoch': 0.22}
{'loss': 0.8651, 'learning_rate': 4.5896032831737345e-05, 'epoch': 0.25}


The following columns in the evaluation set don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: answerC, answerA, answerB, question, context. If answerC, answerA, answerB, question, context are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10023
  Batch size = 16


{'loss': 0.8621, 'learning_rate': 4.544003647970816e-05, 'epoch': 0.27}


Saving model checkpoint to /content/checkpoint/checkpoint-200
Configuration saved in /content/checkpoint/checkpoint-200/config.json


...

Model weights saved in /content/checkpoint/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in /content/checkpoint/checkpoint-2000/tokenizer_config.json
Special tokens file saved in /content/checkpoint/checkpoint-2000/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint/checkpoint-1800] due to args.save_total_limit


{'loss': 0.2561, 'learning_rate': 3.9443684450524395e-06, 'epoch': 2.76}
{'loss': 0.2226, 'learning_rate': 3.488372093023256e-06, 'epoch': 2.79}
{'loss': 0.283, 'learning_rate': 3.032375740994072e-06, 'epoch': 2.82}
{'loss': 0.256, 'learning_rate': 2.5763793889648885e-06, 'epoch': 2.85}
{'loss': 0.2784, 'learning_rate': 2.1203830369357045e-06, 'epoch': 2.87}
{'loss': 0.3112, 'learning_rate': 1.6643866849065208e-06, 'epoch': 2.9}
{'loss': 0.2552, 'learning_rate': 1.208390332877337e-06, 'epoch': 2.93}
{'loss': 0.2319, 'learning_rate': 7.523939808481532e-07, 'epoch': 2.95}
{'loss': 0.2594, 'learning_rate': 2.963976288189695e-07, 'epoch': 2.98}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/checkpoint/checkpoint-1400 (score: 0.6728479862213135).


{'train_runtime': 3184.8548, 'train_samples_per_second': 22.03, 'train_steps_per_second': 0.689, 'train_loss': 0.5212774588754066, 'epoch': 3.0}


TrainOutput(global_step=2193, training_loss=0.5212774588754066, metrics={'train_runtime': 3184.8548, 'train_samples_per_second': 22.03, 'train_steps_per_second': 0.689, 'train_loss': 0.5212774588754066, 'epoch': 3.0})

## Evaluate Model

In [ ]:
model.eval()
test_gths = []; test_preds = []
with torch.no_grad(): 
    for idx, example in tqdm(enumerate(test_dataset)):
        context_question = example['context'] + tokenizer.sep_token + example['question']
        inputs = tokenizer([[context_question, example['answerA']], [context_question, example['answerB']], [context_question, example['answerC']]], padding=True, return_tensors='pt').to(DEVICE)
        labels = int(example['label']) - 1
        outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()})
        logits = outputs.logits
        preds = logits.argmax().item()
        test_gths.append(labels)
        test_preds.append(preds)
        torch.cuda.empty_cache()

0it [00:00, ?it/s]

In [ ]:
accuracy.compute(predictions=test_preds, references=test_gths)

{'accuracy': 0.6038894575230297}

# 과제: [HW5] Multiple choice commonsense reasoning (~ 05/21 23:59)

이번 실습에서는 `social_i_qa` 데이터셋에 대하여 `bert-base` 모델을 학습하고 평가하는 내용을 진행했습니다.  
이에 따라 이번 과제는 `social_i_qa` 및 `commonsense_qa` 데이터셋에 대하여 `roberta-base` 모델을 학습하고 평가하는 것입니다.

```
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForMultipleChoice.from_pretrained('roberta-base')
datasets = load_dataset('commonsense_qa')
```

- 제출 내용
  - Q1) 동일한 조건 하에서 SocialIQA와 CommonsenseQA 성능 비교
    - Q1-a) `social_i_qa` 데이터셋에 학습한 `roberta-base` 모델의 성능을 test set에서 평가하세요. 이때 성능 지표로 accuracy 뿐만 아니라 macro precision, recall, f1-score를 모두 포함해야 합니다. (20점)
      - `social_i_qa` 데이터셋의 경우 train/validation 2가지 split만 존재합니다. 따라서 validation split을 test set으로 사용하도록 합니다.
    - Q1-b) `commonsense_qa` 데이터셋에 대해 학습한 `robert-base` 모델의 성능을 test set에서 평가하세요. 이때 성능 지표로 accuracy 뿐만 아니라 macro precision, recall, f1-score를 모두 포함해야 합니다. (20점)
      - `commonsense_qa` 데이터셋의 경우 train/validation/test 3가지 split이 존재합니다. 따라서 test split을 test set으로 사용하시면 됩니다.
    - Q1-c) 만약 SocialIQA와 CommonsenseQA에서의 성능에 차이가 존재한다면, 그 이유에 대해 서술하세요. (20점)
  - Q2) COMET inference를 활용하여 SocialIQA에서의 성능 개선
    - [`socialiqa` 폴더](https://drive.google.com/drive/folders/17WyMnrNvKKeMatPp4U4AypMlrVMZHR9n?usp=sharing)에는 train/dev(=test) set에 대한 COMET inference (context와 question이 주어졌을 때 생성한 inference) 가 annotation 되어있는 데이터가 relation type 별로 존재합니다. 이를 활용하여 SocialIQA에서의 성능 향상을 이끌어내세요.
    - Q2-a) 실험 환경 (e.g. 학습에 사용한 COMET inference의 종류, 각종 hyperparameters, ...) 을 설명하고, 그러한 세팅을 선택한 이유에 대해 서술하세요. (20점)
    - Q2-b) 실험 결과 (accuracy, macro precision, recall, f1-score) 를 보고하고, 성능이 항샹된 이유에 대하여 서술하세요. (20점)
- 제출 분량
    - 2 page 이상 4 page 이하
- 제출 형식
    - pdf로 변환하여 learnus에 제출
    - 파일명 = HW5_학번_이름.pdf (ex. HW5_2021321394_김채형.pdf)
- 제출 마감 일시
    - 5월 21일 (일) 23:59